In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

Voor een beter weergave voor functies zoals `describe()` van pandas.

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('float_format', '{:f}'.format)
pd.set_option('precision', 2)

Dataset inladen. Alle kolommen gemarkeerd als 'nvt' of '?' worden er uit gegooid.

In [3]:
df_full = pd.DataFrame()
df_full = pd.read_csv("sap_storing_data_hu_project.csv", low_memory=False)

nvt_columns = ["stm_sap_mon_meld_ddt", "stm_mon_begin_ddt", "stm_mon_toelichting_trdl",  "stm_oh_pg_mld",  "stm_geo_mld_uit_functiepl",  "stm_scenario_mon",  "stm_status_melding_sap",  "stm_mon_nr_status_omschr",  "stm_mon_nr__statuscode",  "stm_mon_nr_status_wijzdd",  "stm_aanntpl_ddt",  "stm_geo_gst_uit_functiepl",  "stm_objectdl_code_gst",  "stm_objectdl_groep_gst",  "stm_progfh_in_ddt",  "stm_progfh_in_invoer_ddt",  "stm_progfh_gw_ddt",  "stm_progfh_gw_lwd_ddt",  "stm_progfh_hz", "stm_veroorz_groep", "stm_veroorz_code", "stm_veroorz_tekst_kort", "stm_effect", "stm_afspr_aanvangddt", "stm_mon_eind_ddt", "stm_mon_vhdsincident", "stm_dir_betrok_tr", "stm_aangelegd_dd", "stm_aangelegd_tijd", "stm_mon_begindatum", "stm_mon_begintijd", "stm_progfh_gw_datum", "stm_mon_eind_datum", "stm_mon_eind_tijd", "stm_controle_dd", "stm_akkoord_mon_toewijz", "stm_status_sapnaarmon", "stm_fact_jn", "stm_akkoord_melding_jn", "stm_afsluit_ddt", "stm_afsluit_dd", "stm_afsluit_tijd", "stm_rec_toegev_ddt", "stm_hinderwaarde", "stm_actie", "stm_standplaats", "stm_status_gebr", "stm_wbi_nummer", "stm_projnr", "stm_historie_toelichting", "stm_schade_verhaalb_jn", "stm_schadenr", "stm_schade_status_ga", "stm_schade_statusdatum", "stm_relatiervo_vorig", "stm_relatiervo_volgend", "stm_relatiervo", "stm_afspr_func_hersteldd", "stm_afspr_func_hersteltijd", "stm_sorteerveld", "stm_rapportage_maand", "stm_rapportage_jaar", "stm_x_bron_publ_dt", "stm_x_bron_bestandsnaam", "stm_x_bron_arch_dt", "stm_x_actueel_ind", "stm_x_run_id", "stm_x_bk", "stm_x_start_sessie_dt", "stm_x_vervallen_ind", "stm_evb", "stm_functiepl_mld", "stm_functiepl_gst", "stm_arbeid", "stm_progfh_in_duur", "stm_progfh_gw_lwd_datum", "stm_progfh_gw_lwd_tijd", "stm_progfh_gw_duur", "stm_afspr_aanvangdd", "stm_afspr_aanvangtijd", "stm_reactie_duur"]

for x in nvt_columns:
    df_full.drop(x, axis=1, inplace=True)

Kolommen die ons interessant lijken worden geisoleerd in een nieuwe DataFrame.

In [4]:
df = pd.DataFrame()
df = df_full[["stm_sap_meld_ddt",  "stm_geo_mld",  "stm_equipm_nr_mld",  "stm_equipm_soort_mld",  "stm_km_van_mld",  "stm_km_tot_mld",  "stm_prioriteit",  "stm_geo_gst",  "stm_oorz_code",  "stm_tao_indicator",  "stm_tao_indicator_vorige",  "stm_tao_soort_mutatie",  "stm_tao_telling_mutatie",  "stm_tao_beinvloedbaar_indicator",  "stm_sap_melddatum",  "stm_sap_meldtijd",  "stm_contractgeb_mld",  "stm_techn_mld",  "stm_techn_gst",  "stm_aanngeb_dd",  "stm_aanngeb_tijd",  "stm_aanntpl_dd",  "stm_aanntpl_tijd", "stm_sap_storeinddatum", "stm_sap_storeindtijd", 'stm_fh_tijd', 'stm_fh_duur']]
df.drop_duplicates(keep='last', inplace=True)

C:\python\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
df.describe()

,stm_equipm_nr_mld,stm_km_van_mld,stm_km_tot_mld,stm_prioriteit,stm_oorz_code,stm_tao_telling_mutatie,stm_contractgeb_mld,stm_fh_duur
count,583005.000000,795173.000000,795173.000000,795027.000000,743983.000000,795177.000000,582913.000000,795141.000000
mean,10643704.189654,35294.862427,15037.024811,4.579368,194.505283,0.162546,25.532668,898.933876
std,473729.898591,216796.303713,49336.800170,2.252253,48.598868,0.393651,20.136125,6250.608976
min,10000012.000000,0.000000,0.000000,1.000000,33.000000,-1.000000,0.000000,0.000000
25%,10221276.000000,0.000000,0.000000,2.000000,145.000000,0.000000,9.000000,10.000000
50%,10601265.000000,16300.000000,0.000000,4.000000,214.000000,0.000000,22.000000,82.000000
75%,10957131.000000,44517.000000,11438.000000,5.000000,221.000000,0.000000,34.000000,249.000000
max,30395904.000000,60844493.000000,4221399.000000,9.000000,999.000000,1.000000,99.000000,528297.000000


In [6]:
df['stm_sap_meldtijd'] = pd.to_datetime(df['stm_sap_meldtijd'], errors='coerce')
df['stm_sap_meldtijd'] = df['stm_sap_meldtijd'].dt.floor('30T')
df['stm_sap_meldtijd'].head()

C:\python\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\python\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0                   NaT
1   2019-11-06 09:00:00
2   2019-11-06 12:30:00
3   2019-11-06 16:30:00
4   2019-11-06 22:30:00
Name: stm_sap_meldtijd, dtype: datetime64[ns]

In [7]:
df_reg = pd.DataFrame()
df_reg = df[['stm_geo_mld', 'stm_techn_mld', 'stm_prioriteit', 'stm_sap_meldtijd', 'stm_fh_duur']][df['stm_fh_duur'].between(1, 360)]
df_reg.dropna(inplace=True)
df_reg.head()

,stm_geo_mld,stm_techn_mld,stm_prioriteit,stm_sap_meldtijd,stm_fh_duur
5,614.0,S,9.000000,2019-11-06 11:00:00,7.000000
8,506.0,B,9.000000,2019-11-06 14:00:00,30.000000
22,102.0,S,9.000000,2019-11-06 21:00:00,20.000000
47,43.0,B,9.000000,2019-11-06 18:00:00,15.000000
63,117.0,B,9.000000,2019-11-06 11:30:00,64.000000


In [8]:
df_reg.describe()

,stm_prioriteit,stm_fh_duur
count,394842.000000,394842.000000
mean,3.486929,101.226653
std,1.451838,83.109625
min,1.000000,1.000000
25%,2.000000,38.000000
50%,4.000000,80.000000
75%,5.000000,144.000000
max,9.000000,360.000000


In [9]:
X = df_reg.loc[:, df_reg.columns != 'stm_fh_duur']
y = df_reg['stm_fh_duur']

for column in X.columns:
    X = X.join(pd.get_dummies(X[column]))
    del X[column]
    
X.head()

,0,001,002,004,005,006,007,008,009,010,011,012,013,015,016,017,018,020,021,023,024,025,026,027,028,030,031,033,034,035,036,037,038,039,040,041,042,043,044,046,047,049,050,051,052,053,054,055,056,057,058,059,060,062,063,064,067,070,071,072,073,074,075,076,078,079,080,082,083,084,085,087,088,089,090,091,092,093,095,096,097,098,099,1,1.0,10,10.0,100,100.0,101,101.0,102,102.0,103,103.0,104,104.0,105,105.0,106,107,107.0,108,108.0,109,109.0,11,11.0,110,111,112,112.0,114,114.0,115,115.0,116,116.0,117,117.0,118,118.0,119,119.0,12,12.0,120,120.0,121,121.0,122,122.0,123,123.0,124,124.0,125,125.0,126,126.0,127,127.0,128,128.0,129,129.0,13,13.0,130,130.0,131,131.0,132,132.0,133,133.0,134,134.0,135,135.0,136,136.0,137,137.0,138,138.0,139,139.0,14,143,143.0,144,144.0,145,145.0,146,146.0,149,149.0,15,15.0,152,152.0,155,155.0,158,158.0,16,16.0,161,161.0,163,163.0,164,164.0,165,165.0,166,166.0,17,17.0,18,18.0,19,19.0,2,2.0,20,20.0,200,200.0,201,201.0,203,203.0,204,204.0,205,205.0,206,206.0,208,208.0,209,209.0,21,21.0,210,210.0,211,211.0,212,212.0,213,213.0,216,216.0,217,217.0,22,22.0,226,226.0,228,228.0,23,23.0,24,24.0,25,...,625,625.0,626,626.0,63,63.0,630,630.0,64,64.0,65,65.0,653,653.0,654,654.0,66,66.0,661,661.0,663,663.0,664,664.0,666,666.0,667,67,67.0,68,69,7,7.0,70,70.0,71,71.0,72,73,73.0,74,74.0,75,75.0,751,751.0,756,756.0,76,76.0,777,78,78.0,79,79.0,8,8.0,80,80.0,801,801.0,802,802.0,803,803.0,804,804.0,805,805.0,808,81,810,810.0,811,811.0,814,815,815.0,816,817,817.0,818,818.0,82,82.0,820,820.0,822,822.0,823,823.0,83,83.0,831,837,839,839.0,84,84.0,842,844,85,85.0,850,850.0,851,851.0,852,852.0,855,857,857.0,858,87,87.0,88,88.0,89,89.0,9,9.0,90,90.0,900,901,902,908,908.0,91,91.0,910,912,912.0,913,913.0,916,917,917.0,92,92.0,920,921,922,922.0,923,923.0,924,924.0,925,925.0,926,926.0,927,927.0,928,928.0,93,93.0,935,935.0,937,937.0,938,94,94.0,940,940.0,95,95.0,950,950.0,951,951.0,952,952.0,96,96.0,97,97.0,98,98.0,99,99.0,A,B,E,G,H,I,K,M,O,P,S,T,X,1.0,2.0,4.0,5.0,8.0,9.0,2019-11-06 00:00:00,2019-11-06 00:30:00,2019-11-06 01:00:00,2019-11-06 01:30:00,2019-11-06 02:00:00,2019-11-06 02:30:00,2019-11-06 03:00:00,2019-11-06 03:30:00,2019-11-06 04:00:00,2019-11-06 04:30:00,2019-11-06 05:00:00,2019-11-06 05:30:00,2019-11-06 06:00:00,2019-11-06 06:30:00,2019-11-06 07:00:00,2019-11-06 07:30:00,2019-11-06 08:00:00,2019-11-06 08:30:00,2019-11-06 09:00:00,2019-11-06 09:30:00,2019-11-06 10:00:00,2019-11-06 10:30:00,2019-11-06 11:00:00,2019-11-06 11:30:00,2019-11-06 12:00:00,2019-11-06 12:30:00,2019-11-06 13:00:00,2019-11-06 13:30:00,2019-11-06 14:00:00,2019-11-06 14:30:00,2019-11-06 15:00:00,2019-11-06 15:30:00,2019-11-06 16:00:00,2019-11-06 16:30:00,2019-11-06 17:00:00,2019-11-06 17:30:00,2019-11-06 18:00:00,2019-11-06 18:30:00,2019-11-06 19:00:00,2019-11-06 19:30:00,2019-11-06 20:00:00,2019-11-06 20:30:00,2019-11-06 21:00:00,2019-11-06 21:30:00,2019-11-06 22:00:00,2019-11-06 22:30:00,2019-11-06 23:00:00,2019-11-06 23:30:00
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [11]:
reg = DecisionTreeRegressor(criterion='mse', min_samples_leaf=100)

reg = reg.fit(X_train, y_train)

reg_pred = reg.predict(X_test)

RMSE = np.sqrt(mean_squared_error(y_test, reg_pred))

RMSE

75.64132279516656